# HUMAN FACIAL_EXPRESSION RECOGNITION


7 BASIC HUMAN EXPRESSION:

-HAPPY 

-ANGER

-SAD

-FEARFULL

-DISGUSTED

-SURPRISED

-NEUTRAL


# IMPORTING MODULES   


In [2]:

import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.
C:\Users\vandana poddar\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\vandana poddar\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\vandana poddar\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\vandana poddar\anaconda3\lib\site-packages\t

# Initializing  the training and validation(testing) generators:

In [3]:
train_dir = 'C:/Users/vandana poddar/Desktop/mlops_project/train'
val_dir = 'C:/Users/vandana poddar/Desktop/mlops_project/validation'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


# Building convolution network architecture

In [4]:

emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))




# Compiling and training the model:

In [4]:

emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=7178 // 64)


Epoch 1/30
448/448 [==============================] - 2006s 4s/step - loss: 1.8059 - accuracy: 0.2575 - val_loss: 1.7752 - val_accuracy: 0.3321
Epoch 2/30
448/448 [==============================] - 217s 484ms/step - loss: 1.6397 - accuracy: 0.3589 - val_loss: 1.5939 - val_accuracy: 0.4077
Epoch 3/30
448/448 [==============================] - 233s 520ms/step - loss: 1.5513 - accuracy: 0.4026 - val_loss: 1.5180 - val_accuracy: 0.4359
Epoch 4/30
448/448 [==============================] - 250s 557ms/step - loss: 1.4888 - accuracy: 0.4297 - val_loss: 1.3192 - val_accuracy: 0.4602
Epoch 5/30
448/448 [==============================] - 248s 554ms/step - loss: 1.4265 - accuracy: 0.4565 - val_loss: 1.4320 - val_accuracy: 0.4799
Epoch 6/30
448/448 [==============================] - 243s 543ms/step - loss: 1.3816 - accuracy: 0.4756 - val_loss: 1.4939 - val_accuracy: 0.4853
Epoch 7/30
448/448 [==============================] - 581s 1s/step - loss: 1.3368 - accuracy: 0.4961 - val_loss: 1.4331 - val

# Saving the model weights:

In [5]:
emotion_model.save_weights('emotion_model.h5')

# predict the emotions using webcam

In [6]:
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
        #we have used haarcascade classifier to detect boundingbox of human face in webcam
    bounding_box = cv2.CascadeClassifier('C:/Users/vandana poddar/Desktop/mlops_project/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow('Video', cv2.resize(frame,(360,260),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1)  == 13:
        break
cap.release()
cv2.destroyAllWindows()
